In [ ]:
from itertools import cycle
from random import randrange
from tkinter import Canvas, Tk, font
import random
import pygame


pygame.mixer.init()


background_music = r"c:\Users\User\Downloads\CHILL.mp3"  
win_music = r"c:\Users\User\Downloads\win.mp3"  
lose_music = r"c:\Users\User\Downloads\lo.mp3"  


pygame.mixer.music.load(background_music)
pygame.mixer.music.play(-1)

canvas_width = 1000
canvas_height = 500

root = Tk()
root.title("Egg Catcher")
c = Canvas(root, width=canvas_width, height=canvas_height, background="deep sky blue")
c.create_rectangle(-5, canvas_height - 100, canvas_width + 5, canvas_height + 5, fill="brown", width=0)
c.create_oval(-80, -80, 120, 120, fill='orange', width=0)
c.pack()

colors = ["red", "green", "yellow"]
color_cycle = cycle(colors)
egg_width = 45
egg_height = 55
egg_score = 10
egg_speed = 500
egg_interval = 4000
difficulty = 0.95
catcher_width = 100
catcher_height = 100
catcher_startx = canvas_width / 2 - catcher_width / 2
catcher_starty = canvas_height - catcher_height - 20
catcher_startx2 = catcher_startx + catcher_width
catcher_starty2 = catcher_starty + catcher_height


def create_basket():
    basket_body = c.create_rectangle(
        catcher_startx,
        catcher_starty,
        catcher_startx2,
        catcher_starty2,
        fill="gray",  
        outline="black",
        width=3
    )

    basket_lid = c.create_rectangle(
        catcher_startx,
        catcher_starty - 20,
        catcher_startx2,
        catcher_starty,
        fill="darkgray",  
        outline="black",
        width=3
    )

    return [basket_body, basket_lid]


catcher_parts = create_basket()

game_font = font.nametofont("TkFixedFont")
game_font.config(size=18)

score = 0
score_text = c.create_text(10, 10, anchor="nw", font=game_font, fill="darkblue", text="Score: " + str(score))

lives_remaining = 3
lives_text = c.create_text(canvas_width - 10, 10, anchor="ne", font=game_font, fill="darkblue", text="Lives: " + str(lives_remaining))

requested_color = None
request_text = c.create_text(canvas_width / 2, 10, anchor="n", font=game_font, fill="darkred", text="")

eggs = []
game_won = False
time_left = 60
time_text = c.create_text(canvas_width / 2, canvas_height - 40, font=game_font, fill="darkgreen", text=f"Time: {time_left}s")

# Di chuyển rổ
def move_catcher(dx):
    for part in catcher_parts:
        c.move(part, dx, 0)

def move_left(event):
    (x1, _, _, _) = c.coords(catcher_parts[0])
    if x1 > 0:
        move_catcher(-20)

def move_right(event):
    (_, _, x2, _) = c.coords(catcher_parts[0])
    if x2 < canvas_width:
        move_catcher(20)

def create_egg():
    if game_won or time_left <= 0 or lives_remaining <= 0:
        return
    x = randrange(10, 740)
    y = 40
    new_egg = c.create_oval(x, y, x + egg_width, y + egg_height, fill=next(color_cycle), width=0)
    eggs.append(new_egg)
    root.after(egg_interval, create_egg)

def move_eggs():
    if game_won or time_left <= 0 or lives_remaining <= 0:
        return
    for egg in eggs:
        (eggx, eggy, eggx2, eggy2) = c.coords(egg)
        c.move(egg, 0, 20)
        if eggy2 > canvas_height:
            egg_dropped(egg)
    root.after(egg_speed, move_eggs)

def egg_dropped(egg):
    global requested_color
    egg_color = c.itemcget(egg, "fill")
    eggs.remove(egg)
    c.delete(egg)

    if requested_color and egg_color == requested_color:
        lose_a_life()

    if lives_remaining == 0 and not game_won:
        show_lose_message()
        return

def handle_keypress(event):
    global requested_color
    if event.char.lower() == "r":
        request_color("red")
    elif event.char.lower() == "g":
        request_color("green")
    elif event.char.lower() == "y":
        request_color("yellow")

def lose_a_life():
    global lives_remaining
    lives_remaining -= 1
    c.itemconfigure(lives_text, text="Lives: " + str(lives_remaining))

def increase_score(points):
    global score, egg_speed, egg_interval, game_won
    score += points
    egg_speed = int(egg_speed * difficulty)
    egg_interval = int(egg_interval * difficulty)
    c.itemconfigure(score_text, text="Score: " + str(score))

    if score >= 100 and not game_won:
        game_won = True
        show_win_message()
        return

def request_color(color):
    global requested_color
    requested_color = color
    c.itemconfigure(request_text, text=f"Catch color: {color}")

def check_requested_catch():
    if game_won or time_left <= 0 or lives_remaining <= 0:
        return
    global requested_color
    (catcherx, catchery, catcherx2, catchery2) = c.coords(catcher_parts[0])
    for egg in eggs:
        (eggx, eggy, eggx2, eggy2) = c.coords(egg)
        egg_color = c.itemcget(egg, "fill")
        
        
        if catcherx < eggx and eggx2 < catcherx2 and catchery - eggy2 < 40:
            if requested_color and egg_color == requested_color:
                increase_score(egg_score)
                c.itemconfigure(request_text, text=f"Good job! +{egg_score}")
            else:
                c.itemconfigure(request_text, text="Wrong color! No points.")
            eggs.remove(egg)
            c.delete(egg)
            requested_color = None  
    root.after(100, check_requested_catch)

def countdown():
    global time_left
    if time_left > 0:
        time_left -= 1
        c.itemconfigure(time_text, text=f"Time: {time_left}s")
        root.after(1000, countdown)
    else:
        if not game_won:
            show_lose_message()

def show_win_message():
    c.create_text(canvas_width / 2, canvas_height / 2, text="YOU WIN!", font=game_font, fill="gold", anchor="center")
    pygame.mixer.music.load(win_music)  
    pygame.mixer.music.play(0, 0.0)  
    c.after(3000, root.quit)

def show_lose_message():
    c.create_text(canvas_width / 2, canvas_height / 2, text="YOU LOSE!", font=game_font, fill="red", anchor="center")
    pygame.mixer.music.load(lose_music)  
    pygame.mixer.music.play(0, 0.0) 
    c.after(3000, root.quit)

c.bind("<Left>", move_left)
c.bind("<Right>", move_right)
c.bind("<KeyPress>", handle_keypress)
c.focus_set()

root.after(1000, create_egg)
root.after(1000, move_eggs)
root.after(1000, check_requested_catch)
root.after(1000, countdown)
root.mainloop()
pygame.mixer.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.13.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 